<a href="https://colab.research.google.com/github/woodsj1206/Train-Yolov8-OBB-Object-Detection-On-Custom-Dataset/blob/main/Train_Yolov8_OBB_Object_Detection_On_Custom_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Step 1: Checking GPU Availability
Check if a GPU is available in your Google Colab environment. A GPU can significantly accelerate the training process of deep learning models like YOLOv8.

In [ ]:
!nvidia-smi

#Step 2: Unzipping Dataset Files
Unzip the dataset files containing images and annotations. These files are typically compressed to save space and are now being extracted for use in training the YOLOv8 model.

**NOTE:** Ensure that the images and their corresponding annotation files have the same names (e.g., image_1.png should have an associated image_1.txt file).

In [ ]:
# Unzip the zip file containing the images
!unzip -q 'REPLACE_WITH_YOUR_IMAGE_ZIP_FILE_PATH' -d '/content/images'

# Unzip the zip file containing the annotations
!unzip -q 'REPLACE_WITH_YOUR_ANNOTATIONS_ZIP_FILE_PATH' -d '/content/annotations'

#Step 3: Installing Ultralytics
Install Ultralytics, a library that simplifies working with YOLO object detection models.

In [ ]:
# Install Ultralytics library
!pip install ultralytics

# Import necessary modules from Ultralytics
from ultralytics import YOLO

#Step 4: Mounting Google Drive
Mount Google Drive to the Colab environment, enabling access to files stored in your Google Drive.

In [ ]:
# Import the drive module from google.colab
from google.colab import drive

# Mount Google Drive to '/content/Google_Drive'
drive.mount('/content/Google_Drive')

#Step 5: Define Root Directory and Create Subdirectories
Define the root directory for your project and create necessary subdirectories to organize your data.

In [ ]:
import os

# Prompt user for the path
PATH = input("Enter the desired path: ")

ROOT_DIR = f'/content/Google_Drive/MyDrive/{PATH}'

# Create directory for data
DATA_DIR = os.path.join(ROOT_DIR, 'data')

# Create directory for images
IMAGES_DIR = os.path.join(DATA_DIR, 'images')
IMAGES_VAL_DIR = os.path.join(IMAGES_DIR, 'val')
IMAGES_TRAIN_DIR = os.path.join(IMAGES_DIR, 'train')

# Create directory for labels
LABELS_DIR = os.path.join(DATA_DIR, 'labels')
LABELS_VAL_DIR = os.path.join(LABELS_DIR, 'val')
LABELS_TRAIN_DIR = os.path.join(LABELS_DIR, 'train')

# Create directory for testing
TESTING_DIR = os.path.join(ROOT_DIR, 'testing')

# Create the directories if the root directory doesn't exist
if not os.path.exists(ROOT_DIR):
    os.makedirs(ROOT_DIR)

    os.makedirs(DATA_DIR)

    os.makedirs(IMAGES_DIR)
    os.makedirs(IMAGES_VAL_DIR)
    os.makedirs(IMAGES_TRAIN_DIR)

    os.makedirs(LABELS_DIR)
    os.makedirs(LABELS_VAL_DIR)
    os.makedirs(LABELS_TRAIN_DIR)

    os.makedirs(TESTING_DIR)

    print(f"Root directory '{ROOT_DIR}' created successfully.")
else:
    print(f"Root directory '{ROOT_DIR}' already exists.")

#Step 6: Organize Dataset for Training and Validation
Organize the dataset by moving images and corresponding annotations into separate directories for training and validation.

In [ ]:
import shutil

# Replace placeholders with actual paths to your images and annotations
IMAGES_PATH = 'REPLACE_WITH_PATH_TO_YOUR_IMAGES'
ANNOTATIONS_PATH = 'REPLACE_WITH_PATH_TO_YOUR_ANNOTATIONS'

# Get list of image and annotation files
image_files = os.listdir(IMAGES_PATH)
annotation_files = os.listdir(ANNOTATIONS_PATH)

# Sort the files for consistency
image_files.sort()
annotation_files.sort()

# Determine the number of files for the training set
train_count = int(len(image_files) * 0.7) # 70% of data used for training, 30% used for validation

# Move files to the training directory
for file in image_files[:train_count]:
    shutil.move(os.path.join(IMAGES_PATH, file), os.path.join(IMAGES_TRAIN_DIR, file))
for file in annotation_files[:train_count]:
    shutil.move(os.path.join(ANNOTATIONS_PATH, file), os.path.join(LABELS_TRAIN_DIR, file))

# Move files to the validation directory
for file in image_files[train_count:]:
    shutil.move(os.path.join(IMAGES_PATH, file), os.path.join(IMAGES_VAL_DIR, file))
for file in annotation_files[train_count:]:
    shutil.move(os.path.join(ANNOTATIONS_PATH, file), os.path.join(LABELS_VAL_DIR, file))

#Step 7: Generate YAML Configuration File
Create a YAML configuration file specifying the paths to the training and validation datasets, as well as the class names (labels) used in your dataset.

In [ ]:
import yaml

# Define the data structure for the YAML file
data = {
    'path': f'{DATA_DIR}',
    'train': 'images/train',
    'val': 'images/val',
    'names': {
        # Add your labels here
        0: 'REPLACE_WITH_YOUR_LABEL_NAME'
      # 1: ''
      # 2: ''
      # ...
    }
}

# Construct the full path to the YAML file
output_file = os.path.join(ROOT_DIR, "config.yaml")

# Write the data to the YAML file
with open(output_file, 'w') as yaml_file:
    yaml.dump(data, yaml_file)

#Step 8: Train the YOLOv8 Model
Train the YOLOv8 model using the provided dataset configuration and save the training results.

In [ ]:
# Load a pre-trained YOLOv8 model
model = YOLO("yolov8n-obb.pt")

# Train the model using the provided dataset configuration
model_results = model.train(data=os.path.join(ROOT_DIR, "config.yaml"), epochs=20)

# Save the training results
shutil.make_archive(base_dir='/content/runs', root_dir='/content/runs', format='zip', base_name=f'{ROOT_DIR}/runs')

#Step 9: View Training Results
Display the training results and the confusion matrix generated during the training process.

In [ ]:
from IPython.display import Image

# Display the training results
Image('/content/runs/obb/train/results.png')

In [ ]:
# Display the confusion matrix
Image('/content/runs/obb/train/confusion_matrix.png')

#Step 10: Process Video Files
Process video files using the trained YOLOv8 model and save the processed videos with bounding boxes and class labels.

**NOTE:** Ensure you have uploaded your video files to the testing folder before proceeding with this step.

In [ ]:
import cv2

# Define the directory where you want to save the images
OUTPUT_DIR = '/content/output'

# Create the output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Path to the trained model file
model_file = "/content/runs/obb/train/weights/best.pt"

# Load the trained model
model = YOLO(model_file)

# List all files in the folder
video_files = os.listdir(TESTING_DIR)

# Process each video file
for video_file in video_files:
    # Open the video file
    cap = cv2.VideoCapture(os.path.join(TESTING_DIR, video_file))

    # Get the dimensions and frame rate of the video
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

    # Create the output video writer
    out = cv2.VideoWriter(f'{OUTPUT_DIR}/{video_file}_OBB_object_detection.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

    # Read the first frame of the video
    ret, frame = cap.read()

    while ret:
        # Perform object detection on the frame
        results = model(frame)[0]

        for result in results.obb.data.tolist():
            x_center, y_center, width, height, angle, score, class_id = result

            # Calculate the coordinates of the bounding box corners
            corners = cv2.boxPoints(((x_center, y_center), (width, height), angle * 180 / np.pi))
            corners = np.int0(corners)

            # Draw the bounding box
            cv2.drawContours(frame, [corners], 0, (255, 0, 0), 2)

            # Display class name and score
            label = f'{results.names[int(class_id)]}: {score:.2f}'

            # Get the size of the text
            text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]

            # Draw a filled rectangle behind the text label
            cv2.rectangle(frame, (int(x_center) - 5, int(y_center) - text_size[1] - 10), (int(x_center) + text_size[0] + 5, int(y_center) + 5), (255, 0, 0), cv2.FILLED)

            # Draw the text label
            cv2.putText(frame, label, (int(x_center), int(y_center) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

        out.write(frame)
        ret, frame = cap.read()

    # Release the video capture and writer objects
    cap.release()
    out.release()
    cv2.destroyAllWindows()

#Step 11: Display Processed Video
Use the MoviePy library to display the processed video within the Colab environment.

In [ ]:
from moviepy.editor import *

# Path to the processed video file
path = 'REPLACE_WITH_PATH_TO_VIDEO_FILE'

# Load the processed video clip
clip = VideoFileClip(path)

# Display the video clip
clip.ipython_display(height=540, width=960)